# Capsule Network for RL

In [1]:
import tensorflow as tf
import cv2
import sys
import time 
import random
import numpy as np
from collections import deque
from matplotlib import pyplot as plt
import pygame
from ple import PLE

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


In [2]:
from ple.games.catcher import Catcher
#game = Catcher()
game = Catcher(width=256, height=256)
game.screen = pygame.display.set_mode(game.getScreenDims(), 0, 32)

In [3]:
epsilon = 1e-9
iter_routing = 2
train_freq = 10

In [4]:
GAME  = "CATCHER"
ACTIONS = 3 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVE = 500. # timesteps to observe before training
EXPLORE = 500. # frames over which to anneal epsilon
FINAL_EPSILON =  0.05 # final value of epsilon
INITIAL_EPSILON = 1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1

# UPDATED CAPSULE NETWORK

In [ ]:
def squash(vector):
    vec_squared_norm = reduce_sum(tf.square(vector), -2, keepdims=True)
    scalar_factor = vec_squared_norm / (1 + vec_squared_norm) / tf.sqrt(vec_squared_norm + epsilon)
    vec_squashed = scalar_factor * vector  # element-wise
    return(vec_squashed)
def routing(input, b_IJ):
    # W: [1, num_caps_i, num_caps_j * len_v_j, len_u_j, 1]
    W = tf.get_variable('Weight', shape=(1, 1024, 160, 8, 1), dtype=tf.float32,
                        initializer=tf.random_normal_initializer(stddev=0.01))
    biases = tf.get_variable('bias', shape=(1, 1, 10, 16, 1))
    # A better solution is using element-wise multiply, reduce_sum and reshape
    # ops instead. Matmul [a, b] x [b, c] is equal to a series ops as
    # element-wise multiply [a*c, b] * [a*c, b], reduce_sum at axis=1 and
    # reshape to [a, c]
    input = tf.tile(input, [1, 1, 160, 1, 1])
    #assert input.get_shape() == [cfg.batch_size, 1024, 160, 8, 1]

    u_hat = reduce_sum(W * input, axis=3, keepdims=True)
    u_hat = tf.reshape(u_hat, shape=[-1, 1024, 10, 16, 1])
    #assert u_hat.get_shape() == [cfg.batch_size, 1024, 10, 16, 1]

    # In forward, u_hat_stopped = u_hat; in backward, no gradient passed back from u_hat_stopped to u_hat
    u_hat_stopped = tf.stop_gradient(u_hat, name='stop_gradient')

    # line 3,for r iterations do
    for r_iter in range(iter_routing):
        with tf.variable_scope('iter_' + str(r_iter)):
            # line 4:
            # => [batch_size, 1024, 10, 1, 1]
            c_IJ = softmax(b_IJ, axis=2)

            # At last iteration, use `u_hat` in order to receive gradients from the following graph
            if r_iter == iter_routing - 1:
                # line 5:
                # weighting u_hat with c_IJ, element-wise in the last two dims
                # => [batch_size, 1024, 10, 16, 1]
                s_J = tf.multiply(c_IJ, u_hat)
                # then sum in the second dim, resulting in [batch_size, 1, 10, 16, 1]
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                #assert s_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]

                # line 6:
                # squash using Eq.1,
                v_J = squash(s_J)
                #assert v_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]
            elif r_iter < iter_routing - 1:  # Inner iterations, do not apply backpropagation
                s_J = tf.multiply(c_IJ, u_hat_stopped)
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                v_J = squash(s_J)

                # line 7:
                # reshape & tile v_j from [batch_size ,1, 10, 16, 1] to [batch_size, 1024, 10, 16, 1]
                # then matmul in the last tow dim: [16, 1].T x [16, 1] => [1, 1], reduce mean in the
                # batch_size dim, resulting in [1, 1024, 10, 1, 1]
                v_J_tiled = tf.tile(v_J, [1, 1024, 1, 1, 1])
                u_produce_v = reduce_sum(u_hat_stopped * v_J_tiled, axis=3, keepdims=True)
                #assert u_produce_v.get_shape() == [cfg.batch_size, 1024, 10, 1, 1]

                # b_IJ += tf.reduce_sum(u_produce_v, axis=0, keep_dims=True)
                b_IJ += u_produce_v
    return(v_J)
# For version compatibility
def reduce_sum(input_tensor, axis=None, keepdims=False):
    return tf.reduce_sum(input_tensor, axis=axis, keepdims=keepdims)
# For version compatibility
def softmax(logits, axis=None):
    return tf.nn.softmax(logits, axis=axis)

In [ ]:
def createNetwork():
    # input layer
    s= tf.placeholder("float", [None, 84, 84, 4])
    coeff = tf.placeholder(tf.float32, shape=(None, 1024, 10, 1, 1))
    ####################### New Network COnfiguration #####################    
    w_initializer, b_initializer = tf.random_normal_initializer(0., 0.01), tf.constant_initializer(0.01)
    w1 = tf.get_variable('w1',[8, 8, 4, 64],initializer=w_initializer)
    b1 = tf.get_variable('b1',[64],initializer=b_initializer)
    # Convolution Layer
    # Conv1, [batch_size, 20, 20, 64]
    l1 = tf.nn.conv2d(s, w1, strides=[1, 4, 4, 1], padding="VALID")
    
    conv1 = tf.nn.relu(tf.nn.bias_add(l1, b1))
    
    conv1 = tf.reshape(conv1,[-1,20,20,64])
    
    capsules = tf.contrib.layers.conv2d(conv1, 16 * 8, kernel_size=6, stride=2, padding="VALID",
                    activation_fn = tf.nn.relu,
                    weights_initializer = tf.contrib.layers.xavier_initializer(uniform=False),
                    biases_initializer=tf.constant_initializer(0))
    
    capsules = tf.reshape(capsules, (-1, 1024, 8, 1)) #Reshape to(batch_szie, 1152, 8, 1)
    
    capsules = squash(capsules)
    
    input_caps2 = tf.reshape(capsules, shape=(-1, 1024, 1, capsules.shape[-2].value, 1))
    
    caps2 = routing(input_caps2, coeff)
    
    vector_j = tf.reshape(caps2, shape=(-1, 160))
    #print(vector_j)
    q_eval = tf.contrib.layers.fully_connected(vector_j, num_outputs=ACTIONS, activation_fn=None)

    #print(q_eval)
    readout = q_eval
    return s, coeff, readout

In [ ]:
def trainNetwork(s, coeff, readout, sess):
    tick = time.time()
    # define the cost function
    a = tf.placeholder("float", [None, ACTIONS])
    y = tf.placeholder("float", [None])
    readout_action = tf.reduce_sum(tf.multiply(readout, a), reduction_indices = 1)
    cost = tf.reduce_mean(tf.square(y - readout_action))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cost)

    ## open up a game state to communicate with emulator
    # game_state = game.GameState()
    p =  PLE(game,display_screen=True)
    actions = p.getActionSet()
    p.init()
    reward = p.act(p.NOOP)
    
    # store the previous observations in replay memory
    D = deque()
    
    # get the first state by doing nothing and preprocess the image to 80x80x4
    
    action_index =  np.random.randint(0, 3)
    reward = p.act(actions[action_index])
    if(reward != 0):
        if(reward < -1):
            r_0 = -1
        else:
            r_0 = reward
        terminal = 1
        if(np.max(readout_t) > qmax):
            qmax = np.max(readout_t) 
        print("time_step", t,"/SCORE", score,"/rt", r_t,"/ Q_MAX %e" % qmax)
        qmax = -10000
    else:
        terminal = 0
        r_0 = reward
            
    x_t = p.getScreenRGB()
    x_t = cv2.cvtColor(cv2.resize(x_t[:,:-110,:], (84, 84)), cv2.COLOR_BGR2GRAY)
    ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    sess.run(tf.global_variables_initializer())
    # saving and loading networks
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state("saved_networks")
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")
    b_IJ1 = np.zeros((1, 1024, 10, 1, 1)).astype(np.float32) # batch_size=1
    b_IJ2 = np.zeros((BATCH, 1024, 10, 1, 1)).astype(np.float32) # batch_size=BATCH
    epsilon = INITIAL_EPSILON
    t = 0
    score = 0
    qmax = -100000
    tick = time.time()
    while True:
        # choose an action epsilon greedily
        readout_t = readout.eval(feed_dict = {s:s_t.reshape((1,84,84,4)), coeff:b_IJ1})
        
        a_t = np.zeros([ACTIONS])
        action_index = 0
        if random.random() <= epsilon or t <= OBSERVE:
            action_index = random.randrange(ACTIONS)
            a_t[action_index] = 1
        else:
            action_index = np.argmax(readout_t)
            a_t[action_index] = 1

        # scale down epsilon
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        # run the selected action and observe next state and reward
        reward = p.act(actions[action_index])
        if(reward != 0):
            if(reward < -1):
                r_t = -1
            else:
                r_t = reward
            terminal = 1
            if(np.max(readout_t) > qmax):
                qmax = np.max(readout_t) 
            print("time_step", t,"/SCORE", score,"/rt", r_t,"/ Q_MAX %e" % qmax)
            qmax = -10000
        else:
            terminal = 0
            r_t = reward
        # run the selected action and observe next state and reward
        x_t1_colored = p.getScreenRGB()
        x_t1 = cv2.cvtColor(cv2.resize(x_t1_colored[:,:-110,:], (84, 84)), cv2.COLOR_BGR2GRAY)
        ret, x_t1 = cv2.threshold(x_t1, 1, 255, cv2.THRESH_BINARY)
        x_t1 = np.reshape(x_t1, (84, 84, 1))
        s_t1 = np.append(x_t1, s_t[:, :, :3], axis=2)
        # store the transition in D
        D.append((s_t, a_t, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()
        
        # only train if done observing
        if t > OBSERVE and t%train_freq==0:
            # sample a minibatch to train on
            minibatch = random.sample(D, BATCH)

            # get the batch variables
            s_j_batch = [d[0] for d in minibatch]
            a_batch = [d[1] for d in minibatch]
            r_batch = [d[2] for d in minibatch]
            s_j1_batch = [d[3] for d in minibatch]

            y_batch = []
            readout_j1_batch = readout.eval(feed_dict = {s:s_j1_batch, coeff:b_IJ2 })
            #readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
            for i in range(0, len(minibatch)):
                # if terminal only equals reward
                if minibatch[i][4]:
                    y_batch.append(r_batch[i])
                else:
                    y_batch.append(r_batch[i] + GAMMA * np.max(readout_j1_batch[i]))

            # perform gradient step
            train_step.run(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch,
                coeff: b_IJ2})
            loss = cost.eval(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch,
                coeff: b_IJ2})

        # update the old values
        s_t = s_t1
        t += 1
        
        # save progress every 10000 iterations
        if t % 10000 == 0:
            saver.save(sess, 'saved_networks/' + GAME + '-dqn', global_step = t)

        score += reward
        if p.game_over():
            p.reset_game()
            #print("time_step:",t,"score:",score)
            score = 0

In [ ]:
def playGame():
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    s, coeff, readout = createNetwork()
    trainNetwork(s, coeff, readout, sess)

In [ ]:

def main():
    playGame()

if __name__ == "__main__":
    tick = time.time()
    main()
    print("Game_Ends_in Time:",int(time.time() - tick))
    print("____________ END HERE _____________")

Could not find old network weights
time_step 44 /SCORE 0 /rt -1.0 / Q_MAX 6.446280e-02
time_step 77 /SCORE -1.0 /rt 1.0 / Q_MAX 6.446280e-02
time_step 118 /SCORE 0.0 /rt -1.0 / Q_MAX 6.446280e-02
time_step 164 /SCORE -1.0 /rt -1 / Q_MAX 6.446280e-02
time_step 202 /SCORE 0.0 /rt -1.0 / Q_MAX 6.446280e-02
time_step 241 /SCORE -1.0 /rt -1.0 / Q_MAX 6.446280e-02
time_step 277 /SCORE -2.0 /rt 1.0 / Q_MAX 6.446280e-02
time_step 310 /SCORE -1.0 /rt 1.0 / Q_MAX 6.446280e-02
time_step 356 /SCORE 0.0 /rt -1 / Q_MAX 6.446280e-02
time_step 399 /SCORE 0.0 /rt -1.0 / Q_MAX 6.446280e-02
time_step 433 /SCORE -1.0 /rt -1.0 / Q_MAX 6.446280e-02
time_step 479 /SCORE -2.0 /rt -1 / Q_MAX 6.446280e-02
time_step 526 /SCORE 0.0 /rt -1.0 / Q_MAX 6.492008e-02
time_step 562 /SCORE -1.0 /rt -1.0 / Q_MAX 6.327654e-02
time_step 606 /SCORE -2.0 /rt -1 / Q_MAX 6.219716e-02
time_step 644 /SCORE 0.0 /rt -1.0 / Q_MAX 6.108264e-02
time_step 678 /SCORE -1.0 /rt -1.0 / Q_MAX 6.079523e-02
time_step 717 /SCORE -2.0 /rt -1 / 

time_step 5826 /SCORE -1.0 /rt -1.0 / Q_MAX -3.426402e-02
time_step 5862 /SCORE -2.0 /rt -1 / Q_MAX -3.298322e-02
time_step 5904 /SCORE 0.0 /rt -1.0 / Q_MAX -3.189266e-02
time_step 5941 /SCORE -1.0 /rt -1.0 / Q_MAX -3.041726e-02
time_step 5982 /SCORE -2.0 /rt -1 / Q_MAX -3.062721e-02
time_step 6023 /SCORE 0.0 /rt 1.0 / Q_MAX -3.160847e-02
time_step 6069 /SCORE 1.0 /rt -1.0 / Q_MAX -3.473327e-02
time_step 6100 /SCORE 0.0 /rt 1.0 / Q_MAX -3.671205e-02
time_step 6144 /SCORE 1.0 /rt -1.0 / Q_MAX -3.868214e-02
time_step 6178 /SCORE 0.0 /rt 1.0 / Q_MAX -3.936300e-02
time_step 6216 /SCORE 1.0 /rt 1.0 / Q_MAX -3.997451e-02
time_step 6260 /SCORE 2.0 /rt -1 / Q_MAX -4.037826e-02
time_step 6294 /SCORE 0.0 /rt 1.0 / Q_MAX -4.072812e-02
time_step 6337 /SCORE 1.0 /rt 1.0 / Q_MAX -4.117761e-02
time_step 6374 /SCORE 2.0 /rt 1.0 / Q_MAX -4.121467e-02
time_step 6408 /SCORE 3.0 /rt -1.0 / Q_MAX -4.112569e-02
time_step 6445 /SCORE 2.0 /rt -1.0 / Q_MAX -4.093898e-02
time_step 6481 /SCORE 1.0 /rt -1 / Q_MAX

time_step 11408 /SCORE 1.0 /rt 1.0 / Q_MAX -5.773137e-02
time_step 11444 /SCORE 2.0 /rt 1.0 / Q_MAX -5.950546e-02
time_step 11488 /SCORE 3.0 /rt 1.0 / Q_MAX -6.017870e-02
time_step 11521 /SCORE 4.0 /rt 1.0 / Q_MAX -6.206249e-02
time_step 11557 /SCORE 5.0 /rt -1.0 / Q_MAX -6.310072e-02
time_step 11591 /SCORE 4.0 /rt 1.0 / Q_MAX -6.470977e-02
time_step 11632 /SCORE 5.0 /rt -1 / Q_MAX -6.515369e-02
time_step 11675 /SCORE 0.0 /rt -1.0 / Q_MAX -6.553306e-02
time_step 11714 /SCORE -1.0 /rt -1.0 / Q_MAX -6.533998e-02
time_step 11756 /SCORE -2.0 /rt -1 / Q_MAX -6.504700e-02
time_step 11796 /SCORE 0.0 /rt -1.0 / Q_MAX -6.565753e-02
time_step 11838 /SCORE -1.0 /rt -1.0 / Q_MAX -6.426069e-02
time_step 11872 /SCORE -2.0 /rt 1.0 / Q_MAX -6.391793e-02
time_step 11908 /SCORE -1.0 /rt -1 / Q_MAX -6.359251e-02
time_step 11948 /SCORE 0.0 /rt -1.0 / Q_MAX -6.236546e-02
time_step 11994 /SCORE -1.0 /rt -1.0 / Q_MAX -6.074094e-02
time_step 12033 /SCORE -2.0 /rt 1.0 / Q_MAX -5.894309e-02
time_step 12067 /SCO

time_step 16926 /SCORE 1.0 /rt 1.0 / Q_MAX -7.083259e-02
time_step 16959 /SCORE 2.0 /rt 1.0 / Q_MAX -7.181761e-02
time_step 16992 /SCORE 3.0 /rt 1.0 / Q_MAX -7.369763e-02
time_step 17033 /SCORE 4.0 /rt -1.0 / Q_MAX -7.419452e-02
time_step 17079 /SCORE 3.0 /rt -1.0 / Q_MAX -7.440483e-02
time_step 17116 /SCORE 2.0 /rt 1.0 / Q_MAX -7.619636e-02
time_step 17160 /SCORE 3.0 /rt -1 / Q_MAX -7.846510e-02
time_step 17202 /SCORE 0.0 /rt 1.0 / Q_MAX -8.196047e-02
time_step 17248 /SCORE 1.0 /rt -1.0 / Q_MAX -8.364554e-02
time_step 17294 /SCORE 0.0 /rt -1.0 / Q_MAX -8.493941e-02
time_step 17340 /SCORE -1.0 /rt -1 / Q_MAX -8.649185e-02
time_step 17378 /SCORE 0.0 /rt 1.0 / Q_MAX -8.827108e-02
time_step 17419 /SCORE 1.0 /rt -1.0 / Q_MAX -8.961426e-02
time_step 17453 /SCORE 0.0 /rt 1.0 / Q_MAX -9.050734e-02
time_step 17492 /SCORE 1.0 /rt -1.0 / Q_MAX -9.168159e-02
time_step 17538 /SCORE 0.0 /rt -1 / Q_MAX -9.365699e-02
time_step 17585 /SCORE 0.0 /rt -1.0 / Q_MAX -9.446035e-02
time_step 17629 /SCORE -1.

time_step 22601 /SCORE 1.0 /rt 1.0 / Q_MAX -1.133745e-01
time_step 22635 /SCORE 2.0 /rt 1.0 / Q_MAX -1.141716e-01
time_step 22671 /SCORE 3.0 /rt -1.0 / Q_MAX -1.141308e-01
time_step 22707 /SCORE 2.0 /rt 1.0 / Q_MAX -1.129616e-01
time_step 22739 /SCORE 3.0 /rt 1.0 / Q_MAX -1.118708e-01
time_step 22775 /SCORE 4.0 /rt -1.0 / Q_MAX -1.103199e-01
time_step 22816 /SCORE 3.0 /rt 1.0 / Q_MAX -1.096834e-01
time_step 22852 /SCORE 4.0 /rt -1 / Q_MAX -1.082511e-01
time_step 22889 /SCORE 0.0 /rt -1.0 / Q_MAX -1.083582e-01
time_step 22930 /SCORE -1.0 /rt -1.0 / Q_MAX -1.083918e-01
time_step 22974 /SCORE -2.0 /rt -1 / Q_MAX -1.075994e-01
time_step 23014 /SCORE 0.0 /rt 1.0 / Q_MAX -1.059323e-01
time_step 23055 /SCORE 1.0 /rt -1.0 / Q_MAX -1.042134e-01
time_step 23088 /SCORE 0.0 /rt 1.0 / Q_MAX -1.048784e-01
time_step 23126 /SCORE 1.0 /rt 1.0 / Q_MAX -1.054420e-01
time_step 23165 /SCORE 2.0 /rt -1.0 / Q_MAX -1.055699e-01
time_step 23199 /SCORE 1.0 /rt -1 / Q_MAX -1.059492e-01
time_step 23240 /SCORE 0.0

time_step 28142 /SCORE 4.0 /rt 1.0 / Q_MAX -1.221463e-01
time_step 28175 /SCORE 5.0 /rt 1.0 / Q_MAX -1.235843e-01
